In [1]:
import copy
from functools import cache
from pprint import pprint

with open("input.txt", 'r') as f:
    input = f.read()

input1 = """x00: 1
x01: 1
x02: 1
y00: 0
y01: 1
y02: 0

x00 AND y00 -> z00
x01 XOR y01 -> z01
x02 OR y02 -> z02"""

input1 = """x00: 1
x01: 0
x02: 1
x03: 1
x04: 0
y00: 1
y01: 1
y02: 1
y03: 1
y04: 1

ntg XOR fgs -> mjb
y02 OR x01 -> tnw
kwq OR kpj -> z05
x00 OR x03 -> fst
tgd XOR rvg -> z01
vdt OR tnw -> bfw
bfw AND frj -> z10
ffh OR nrd -> bqk
y00 AND y03 -> djm
y03 OR y00 -> psh
bqk OR frj -> z08
tnw OR fst -> frj
gnj AND tgd -> z11
bfw XOR mjb -> z00
x03 OR x00 -> vdt
gnj AND wpb -> z02
x04 AND y00 -> kjc
djm OR pbm -> qhw
nrd AND vdt -> hwm
kjc AND fst -> rvg
y04 OR y02 -> fgs
y01 AND x02 -> pbm
ntg OR kjc -> kwq
psh XOR fgs -> tgd
qhw XOR tgd -> z09
pbm OR djm -> kpj
x03 XOR y03 -> ffh
x00 XOR y04 -> ntg
bfw OR bqk -> z06
nrd XOR fgs -> wpb
frj XOR qhw -> z04
bqk OR frj -> z07
y03 OR x01 -> nrd
hwm AND bqk -> z03
tgd XOR rvg -> z12
tnw OR pbm -> gnj"""

def disect_op(operation: str):
    lhs, out = tuple(operation.split(" -> "))
    operand1, op, operand2 = lhs.split(" ")
    return operand1, op, operand2, out

def disect_in(wire: str):
    name, val = tuple(wire.split(": "))
    return name, int(val)

def vals2dict(wires: list):
    return {x[0]: x[1] for x in wires}

inputs, operations = tuple(input.split("\n\n"))
instructions = [disect_op(o) for o in operations.split("\n")]
wires = vals2dict([disect_in(w) for w in inputs.split("\n")])

# wires, instructions

In [2]:
def XOR(a, b): return a ^ b
def OR(a, b): return a | b
def AND(a,b): return a & b


OP2FUNC = {
    "OR": OR,
    "AND": AND,
    "XOR": XOR,
}

def execute(instruction: tuple) -> None:
    operand1, op, operand2, out = instruction
    WIRES[out] = OP2FUNC[op](WIRES[operand1], WIRES[operand2])

def decode_output(wires: dict, char="z") -> int:
    z_keys = sorted([k for k in wires.keys() if k.startswith(char)])
    zs = [str(wires[k]) for k in z_keys]
    z_binary_str = "".join(reversed(zs))
    return int(z_binary_str, base=2)

def find_unavailable_instructions(instructions: list):
    availables, dones = [], []
    for inst in instructions:
        if inst[3] in WIRES: 
            dones.append(inst)
        elif inst[0] in WIRES and inst[2] in WIRES:
            availables.append(inst)
    unavailables = list(set(instructions) - set(dones) - set(availables))
    return dones, availables, unavailables

def execute_all(instructions: list) -> None:
    _, availables, unavailables = find_unavailable_instructions(instructions)
    while len(unavailables) > 0 or len(availables) > 0:
        # print(f"Availables: {len(availables)}, Un-availables: {len(unavailables)}")
        if len(availables) == 0:
            return -1
        for inst in availables:
            execute(inst)
        _, availables, unavailables = find_unavailable_instructions(unavailables)

WIRES = copy.copy(wires)

execute_all(instructions)
result = decode_output(WIRES, char="z")
result

46362252142374

### Part 2

Unfortunately could not solve this one! :(

Check solve2.ipynb for a borrowed solution. 
Below are my failed attempts at solving the problem.

In [185]:
def compute_desired_output():
    return decode_output(WIRES, "x") + decode_output(WIRES, "y")

def int_2_binary(z: int) -> str:
    z_str = ""
    while  z > 0:
        z_str = str(z%2) + z_str
        z //= 2
    return z_str

# def dependency_dict(instructions: list):
DD = {out: [operand1, operand2] for (operand1, _, operand2, out) in instructions}

@cache
def explore_dependency(variable: str):
    if DD.get(variable, -1) == -1: return set()
    reqs = set()
    for var in DD[variable]:
        reqs.add(var)
        reqs = reqs.union(explore_dependency(var))
    # print(variable, reqs)
    return reqs


desired_z = compute_desired_output()
print(desired_z)


print(int_2_binary(result))
print(int_2_binary(desired_z))

## Which instructions are involved in the errors
bin_result = int_2_binary(result)
bin_desired = int_2_binary(desired_z)
discrepancies = [r!=d for r,d in zip(bin_result, bin_desired)]
rs = set()
for i, d in enumerate(discrepancies):
    if d == True:
        varname = "z"+str(len(discrepancies)-1-i).zfill(2)
        print(varname)
        rs = rs.union(explore_dependency(varname))
len(rs)

explore_dependency("z01")

46087374243558
1010100010101010001101100100011110011100100110
1010011110101010001101100100100000011011100110
z41
z40
z39
z38
z17
z16
z15
z14
z13
z08
z07
z06


{'fht', 'nqp', 'x00', 'x01', 'y00', 'y01'}

In [186]:
def fill_name(char: str, i: int):
    return f"{char}{str(i).zfill(2)}"
# create our own addition
W = copy.copy(wires)

def check_correct(wires_to_test) -> int:
    i = 0
    W['c01'] = OP2FUNC["AND"](W['x00'], W['y00'])
    W['z00'] = OP2FUNC['XOR'](W['x00'], W['y00'])
    if W['z00'] != wires_to_test['z00']: return 0
    for i in range(1, 45):
        # x XOR y will be caled "q"
        W[fill_name("q",i)] = OP2FUNC['XOR'](W[fill_name("x",i)], W[fill_name("y",i)])
        W[fill_name("a",i)] = OP2FUNC['AND'](W[fill_name("x",i)], W[fill_name("y",i)])
        W[fill_name("z",i)] = OP2FUNC['XOR'](W[fill_name("c",i)], W[fill_name("q",i)])
        # RHS of ci
        W[fill_name("r",i)] = OP2FUNC['AND'](W[fill_name("c",i)], W[fill_name("q",i)])
        W[fill_name("c",i+1)] = OP2FUNC['OR'](W[fill_name("a",i)], W[fill_name("r",i)])

        if W[fill_name("z",i)] != wires_to_test[fill_name("z",i)]:
            return i

    W['z45'] = W['c45']
    if W['z45'] != wires_to_test['z45']: return 45 
    z_out = decode_output(W, "z")
    return 46

def swap_instructions(inst, i, j):
    # global instructions
    instructions = copy.copy(inst)
    operand1i, opi, operand2i, outi = instructions[i]
    operand1j, opj, operand2j, outj = instructions[j]
    instructions[i] = operand1i, opi, operand2i, outj
    instructions[j] = operand1j, opj, operand2j, outi
    return instructions

# INSTS = copy.copy(instructions)
swapped_indices = []
progress = check_correct(WIRES)
INSTS = copy.copy(instructions)

print(instructions[39], instructions[60])

for swapi in range(4):
    for swap in swapped_indices:
        INSTS = swap_instructions(INSTS, i,j)
        WIRES = copy.copy(wires)
        result = execute_all(new_instructions)

    found  = False
    for i, inst1 in enumerate(INSTS):
        if i%50 == 0: print(f"i={i}/{len(INSTS)}")
        for j, inst2 in enumerate(INSTS):
            # INSTS[i], INSTS[j] = INSTS[j], INSTS[i]
            new_instructions = swap_instructions(INSTS, i,j)
            WIRES = copy.copy(wires)
            result = execute_all(new_instructions)
            
            
            if result == -1: continue

            progress_after = check_correct(WIRES)
            if i==39 and j==60:
                # pprint(new_instructions)
                print(progress_after)
                # raise ValueError
            if progress_after == progress:
                best_indices_so_far.append((i,j))
            if progress_after > progress:
                found = True
                print(f"Improved progress from {progress} to {progress_after}")
                best_indices_so_far = [(i,j)]
                # best_instructions_so_far = new_instructions
                progress = progress_after
                
            # else:
                # swap back
                # INSTS = swap_instructions(INSTS, i, j)
    if found:
        # INSTS = best_instructions_so_far
        swapped_indices.append(best_indices_so_far)

    # if progress == 46: break
    

print(swapped_indices)


si1 = swapped_indices[::2]
si2 = swapped_indices[1::2]
import itertools

for comb in itertools.combinations(range(len(swapped_indices)), 4):
    # sww2 = [(s1, s2) for s1,s2 in sww[comb]]
    INSTS = copy.copy(instructions)
    WIRES = copy.copy(wires)
    execute_all(INSTS)
    print(check_correct(WIRES))
    WIRES = copy.copy(wires)
    all_swaps = []
    for i in comb:
        s1, s2 = swapped_indices[i]
        all_swaps.append((s1, s2))
        INSTS = swap_instructions(INSTS, s1, s2)
    WIRES = copy.copy(wires)
    result = execute_all(INSTS)
    if result == -1: continue
    progress = check_correct(WIRES)
    print(all_swaps, progress)
    print(decode_output(WIRES, "z"))
    


('x06', 'AND', 'y06', 'z06') ('cjt', 'XOR', 'sfm', 'jmq')
i=0/222
Improved progress from 6 to 7
Improved progress from 7 to 13
13
i=50/222
i=100/222


KeyboardInterrupt: 